In [1]:
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Data Source:https://www.kaggle.com/datasets/sarahquesnelle/canada-data
Temperature_DF = pd.read_csv("Resources/Canada_Temperature_Data.csv")

Temperature_DF2 = Temperature_DF.loc[(Temperature_DF["Year"] >= 1989) & (Temperature_DF["Year"] <= 2020), :]

Prov_DF = Temperature_DF2.loc[Temperature_DF2["Prov"].isin(["ON", "AB", "QC"]), :]

Prov_DF2 = Prov_DF.drop("Stn_Name", axis=1)

Prov_DF2


# Prov_DF2.to_csv("Canada_Temperature_Data2.csv", index=False)


,Year,Month,Prov,Tm,S,P
941030,1989,1,AB,-10.4,27.0,35.5
941031,1989,1,AB,-11.1,24.0,28.0
941032,1989,1,AB,-12.8,37.5,43.5
941033,1989,1,AB,-9.1,36.0,54.0
941034,1989,1,AB,-12.9,20.2,23.2
...,...,...,...,...,...,...
1357243,2017,9,QC,13.5,0.0,62.4
1357244,2017,9,QC,14.5,0.0,83.8
1357245,2017,9,QC,14.1,0.0,52.2
1357246,2017,9,QC,9.3,0.0,76.4


In [4]:
Canada_Temperature_DF = pd.read_csv("Resources/Canada_Temperature_Data2.csv")


Canada_Temperature_DF['Year'] = Canada_Temperature_DF['Year'].astype(str)


Canada_Temperature_DF.head(10)

# Notes

## Average temperature recorded over a given Month. (in Celsius)
## The total Snowfall recorded for a given Month. (in centimeters)
## The total precipitation recorded over a given month. (in millimeters)

,Province,Year,Month,Seasons,Average Temperature,Average Snowfall Recorded,Average Precipitation Recorded
0,AB,1989.0,Jan,Winter,-10.41,35.57,36.57
1,AB,1989.0,Feb,Winter,-15.00,16.96,16.23
2,AB,1989.0,Mar,Winter,-8.81,17.71,17.61
3,AB,1989.0,Apr,Spring,4.80,8.56,16.32
4,AB,1989.0,May,Spring,8.64,16.10,60.84
5,AB,1989.0,Jun,Spring,13.65,0.75,71.83
6,AB,1989.0,Jul,Summer,16.45,0.00,83.52
7,AB,1989.0,Aug,Summer,15.06,0.49,100.48
8,AB,1989.0,Sep,Summer,10.20,1.09,34.99
9,AB,1989.0,Oct,Autumn,4.38,5.89,19.69


In [49]:
# group by year, season, and province
grouped = Canada_Temperature_DF.groupby(['Year', 'Seasons', 'Province'])

# calculate summary statistics
summary_stats = grouped.agg({'Average Temperature': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Snowfall Recorded': ['mean', 'median', 'std', 'var', 'sem'],
                             'Average Precipitation Recorded': ['mean', 'median', 'std', 'var', 'sem']})

# flatten multi-level column names
summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]

# reset index
summary_stats = summary_stats.reset_index()

# print summary statistics by year, season, and province
print(summary_stats)



       Year Seasons     Province  Average Temperature_mean  \
0    1989.0  Autumn           AB                 -2.196667   
1    1989.0  Autumn           BC                  3.343333   
2    1989.0  Autumn           ON                 -1.750000   
3    1989.0  Spring           AB                  9.030000   
4    1989.0  Spring           BC                 11.196667   
..      ...     ...          ...                       ...   
344  2017.0  Summer           ON                 18.530000   
345  2017.0  Winter           AB                 -7.510000   
346  2017.0  Winter           BC                 -0.410000   
347  2017.0  Winter           ON                 -3.233333   
348     nan    #N/D  Total Geral                  6.360000   

     Average Temperature_median  Average Temperature_std  \
0                         -3.28                 6.107490   
1                          2.34                 3.514119   
2                         -0.53                11.020762   
3              

In [50]:
# save summary_stats to Excel file
summary_stats.to_excel('summary_stats.xlsx', index=False)